In [ ]:
rm(list= ls())
library(dplyr)
library(Seurat)
library(patchwork)
library(monocle3)
library(scWGCNA)
filelist = list.files("/data/dk/ischematic_stroke/rawdata")
filelist

para1 = Read10X('~/dk/thyroid/raw/para1/')
para2 = Read10X('~/dk/thyroid/raw/para2/')
para3 = Read10X('~/dk/thyroid/raw/para3/')
para5 = Read10X('~/dk/thyroid/raw/para5/')
para8 = Read10X('~/dk/thyroid/raw/para8/')
para9 = Read10X('~/dk/thyroid/raw/para9/')
tumor1 = Read10X('~/dk/thyroid/raw/tumor1/')
tumor2 = Read10X('~/dk/thyroid/raw/tumor2/')
tumor3 = Read10X('~/dk/thyroid/raw/tumor3/')
tumor5 = Read10X('~/dk/thyroid/raw/tumor5/')
tumor8 = Read10X('~/dk/thyroid/raw/tumor8/')
tumor9 = Read10X('~/dk/thyroid/raw/tumor9/')

p1 <- CreateSeuratObject(counts = para1, project = "para1" , min.cells = 3,min.features = 200)
p2 <- CreateSeuratObject(counts = para2, project = "para2" , min.cells = 3,min.features = 200)
p3 <- CreateSeuratObject(counts = para3, project = "para3" , min.cells = 3,min.features = 200)
p5 <- CreateSeuratObject(counts = para5, project = "para5" , min.cells = 3,min.features = 200)
p8 <- CreateSeuratObject(counts = para8, project = "para8" , min.cells = 3,min.features = 200)
p9 <- CreateSeuratObject(counts = para9, project = "para9" , min.cells = 3,min.features = 200)
t1 <- CreateSeuratObject(counts = tumor1, project = "tumor1" , min.cells = 3,min.features = 200)
t2 <- CreateSeuratObject(counts = tumor2, project = "tumor2" , min.cells = 3,min.features = 200)
t3 <- CreateSeuratObject(counts = tumor3, project = "tumor3" , min.cells = 3,min.features = 200)
t5 <- CreateSeuratObject(counts = tumor5, project = "tumor5" , min.cells = 3,min.features = 200)
t8 <- CreateSeuratObject(counts = tumor8, project = "tumor8" , min.cells = 3,min.features = 200)
t9 <- CreateSeuratObject(counts = tumor9, project = "tumor9" , min.cells = 3,min.features = 200)

obj <- merge(p1,y = c(p2,p3,p5,p8,p9,t1,t2,t3,t5,t8,t9), add.cell.ids=c("p1","p2","p3","p5","p8","p9","t1","t2","t3","t5","t8","t9"))
rm(p1,p2,p3,p5,p8,p9,t1,t2,t3,t5,t8,t9)


obj[["percent.mt"]] <- PercentageFeatureSet(obj, pattern = "^MT-")
obj <- NormalizeData(obj, normalization.method = "LogNormalize", scale.factor = 10000)
obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(obj)
obj <- ScaleData(obj, features = all.genes)
obj <- RunPCA(obj, features = VariableFeatures(object = obj))
ElbowPlot(obj)
obj <- FindNeighbors(obj, dims = 1:15)
obj <- FindClusters(obj, resolution = 0.3)
obj <- RunUMAP(obj, dims = 1:15)

groupcol = ifelse(grepl("para", obj$orig.ident), "para","tumor" )
obj <- AddMetaData(obj, metadata = groupcol, col.name = "group")
DimPlot(object = obj, reduction = 'umap', group.by = 'group')
obj.markers <- FindAllMarkers(obj, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 1)





table(obj.markers$cluster)
allmarker = obj.markers %>%
    group_by(cluster) %>%
    slice_max(n = 10, order_by = avg_log2FC)
for (i in 0:22){
  cat("\n\n")
  print(i)
  cat(allmarker %>% filter(cluster==i) %>% pull(gene) %>% paste(collapse = ','))
}
all = data.frame(obj.markers)
schwann =  all %>% filter(cluster == 'Schwann cell') %>% pull(gene)
res1 = obj.markers %>% filter(cluster == 'Schwann cell')

VlnPlot(obj, features = c("CD79A","CD74","CD79B","IGKC1","MS4A1"), slot = "counts", log = TRUE)
VlnPlot(obj, features = c("CD31","CLDN5","ERG","ID1","MGP","RAMP2","TFP1","TIMP3"), slot = "counts", log = TRUE)
VlnPlot(obj, features = c("CDH1","CLU","DIO2","EPCAM","FN1","ID4","IYD","KRT18","MGST1","S100A13"), slot = "counts", log = TRUE)


obj = RenameIdents(object = obj,'2'='B cell')
obj = RenameIdents(object = obj,'4'='Myeloid cell')
obj = RenameIdents(object = obj,'6'='follicular cell')
obj = RenameIdents(object = obj,'7'='Epithelial cell')
obj = RenameIdents(object = obj,'8'='Epithelial cell')
obj = RenameIdents(object = obj,'9'='Filbroblast cell')
obj = RenameIdents(object = obj,'11'='Endothelial cell')
obj = RenameIdents(object = obj,'12'='Epithelial cell')
obj = RenameIdents(object = obj,'13'='B cell')
obj = RenameIdents(object = obj,'14'='Epithelial cell')
obj = RenameIdents(object = obj,'15'='Endothelial cell')
obj = RenameIdents(object = obj,'17'='Myeloid cell')
obj = RenameIdents(object = obj,'20'='Myeloid cell')
obj = RenameIdents(object = obj,'21'='follicular cell')
obj = RenameIdents(object = obj,'22'='Filbroblast cell')
obj = RenameIdents(object = obj,'0'='Others')
obj = RenameIdents(object = obj,'1'='Others')
obj = RenameIdents(object = obj,'3'='Others')
obj = RenameIdents(object = obj,'5'='Others')
obj = RenameIdents(object = obj,'10'='Others')
obj = RenameIdents(object = obj,'16'='Others')
obj = RenameIdents(object = obj,'18'='Others')
obj = RenameIdents(object = obj,'19'='Others')

DimPlot(object = obj, reduction = 'umap',label = TRUE)


obj = readRDS("~/dk/thyroid/thyroid_obj.RDS")

thyroidmarker = FindMarkers(obj ,logfc.threshold = 1, ident.1 = "tumor", ident.2 = 'para', group.by = 'group', subset.ident = "Thyrocytes cells")
head(thyroidmarker)

thyroid = subset(obj, idents = 'Thyrocytes cells')
express = data.frame(AverageExpression(thyroid, group.by = 'group'))
express[1:3,1:2]
diffexpress = merge(express, thyroidmarker, by ="row.names")
head(diffexpress)

library('ggpubr')
colnames(diffexpress)[4]  = 'padj'
colnames(diffexpress)[5]  = 'log2FoldChange'

diffexpress$baseMean = (diffexpress$RNA.para+diffexpress$RNA.tumor)/2
ggmaplot(diffexpress, size= 1.5, fdr = 0.05, fc =1,top = 20)

saveRDS(obj,'obj.RDS')

obj = readRDS('obj.RDS')




#WGCNA

astro <- merge(t1,y = c(t2,t3,t5,t8,t9),add.cell.ids=c("tumor1","tumor2","tumor3","tumor5","tumor8","tumor9"))
astro[["percent.mt"]] <- PercentageFeatureSet(astro, pattern = "^MT-")
astro <- NormalizeData(astro, normalization.method = "LogNormalize", scale.factor = 10000)
astro <- FindVariableFeatures(astro, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(astro)
astro <- ScaleData(astro, features = all.genes)
astro <- RunPCA(astro, features = VariableFeatures(object = astro))
ElbowPlot(astro)
astro <- FindNeighbors(astro, dims = 1:15)
astro <- FindClusters(astro, resolution = 0.3)
astro <- RunUMAP(astro, dims = 1:15)
astro = RunTSNE(object = astro,dims.use = 1:15)
DimPlot(astro, reduction = "tsne")
MmLimbE155.pcells = calculate.pseudocells(s.cells = astro, # Single cells in Seurat object
                                          seeds=0.2, # Fraction of cells to use as seeds to aggregate pseudocells
                                          nn = 10, # Number of neighbors to aggregate
                                          reduction = "pca", # Reduction to use
                                          dims = 1:15) # The dimensions to use
MmLimbE155.scWGCNA = run.scWGCNA(p.cells = astro, # Pseudocells (recommended), or Seurat single cells
                                 s.cells = astro, # single cells in Seurat format
                                 is.pseudocell = F, # We are using single cells twice this time
                                 features = VariableFeatures(astro))
png("~/dk/thyroid/figures/wgcna.png",width = 500,height = 400)
scW.p.dendro(scWGCNA.data = MmLimbE155.scWGCNA)
dev.off()
scW.p.expression(s.cells = my.small_MmLimbE155, # Single cells in Seurat format
                 scWGCNA.data = MmLimbE155.scWGCNA, # scWGCNA list dataset
                 modules = 11, # Which modules to plot?
                 reduction = "tsne", # Which reduction to plot?
                 ncol=3) 
MmLimbE155.scWGCNA = scWGCNA.networks(scWGCNA.data = MmLimbE155.scWGCNA)
scW.p.network(MmLimbE155.scWGCNA, module=11)
#get intersect genes for pathway analysis

length(diffexpress$Row.names)
inter <- c()
for (i in names(MmLimbE155.scWGCNA$modules)) {interset  = length(intersect(rownames(MmLimbE155.scWGCNA$modules[[i]]), diffexpress$Row.names));inter = append(inter,interset);print(interset)}
names(inter) = paste('module',seq(1:12))
intersect_bar = data.frame(module = paste('module',seq(1:12)),internumber = inter)
intersect_bar$module = reorder(intersect_bar$module,intersect_bar$module)
ggplot(intersect_bar,aes(x=module,y=internumber,fill = 'module'))+geom_bar(stat = 'identity')+theme(legend.position = "none")

inge = intersect(rownames(MmLimbE155.scWGCNA$modules[[11]]),diffexpress$Row.names)
cat(inge)
intersect_matrix = diffexpress[diffexpress$Row.names %in% inge,]
write.table(intersect_matrix,file = '~/dk/thyroid/network_matrx.txt',quote = FALSE,sep = '\t',row.names = FALSE)


library(clusterProfiler)
library(org.Hs.eg.db)
go = enrichGO(gene =  inge, OrgDb = org.Hs.eg.db, ont = 'ALL', pAdjustMethod = 'BH', pvalueCutoff = 0.01, qvalueCutoff = 0.05)
kegg = enrichKEGG(gene =  inge, organism = 'hsa', keyType = 'SYMBOL',pvalueCutoff = '0.05')

dotplot(go)


#trajacet analysis
library(SeuratWrappers)

monobj <- as.cell_data_set(obj)
mono_tumor <- monobj[, monobj$orig.ident %in% c("tumor1","tumor2","tumor3","tumor5","tumor8","tumor9")]

mono_tumor <- cluster_cells(cds = mono_tumor, reduction_method = "UMAP")
mono_tumor <- learn_graph(mono_tumor, use_partition = TRUE)
root_group = colnames(subset(mono_tumor, monobj$indent == "Thyrocytes cell"))
mono_tumor <- order_cells(mono_tumor, reduction_method = "UMAP",root_cells = root_group)
mono_tumor <- order_cells(mono_tumor, reduction_method = "UMAP")

VlnPlot(obj, features = c('FN1','LRRK2','TSC22D1','CCND1','PRDX1','CLU','APOE','DSP','LGALS3','EPS8','MET','ANXA1','CRYAB'))
                          
FeaturePlot(obj, features = c('FN1','LRRK2','TSC22D1','CCND1','PRDX1','CLU','APOE','DSP','LGALS3','EPS8','MET','ANXA1','CRYAB'),ncol = 3)
ciliated_genes <- c('FN1','LRRK2','TSC22D1','CCND1','PRDX1','CLU','APOE','DSP','LGALS3','EPS8','MET','ANXA1','CRYAB')
rowData(mono_tumor)$gene_short_name <-row.names(rowData(mono_tumor))
plot_cells(mono_tumor)
plot_cells(mono_tumor,genes=ciliated_genes,label_cell_groups=FALSE,show_trajectory_graph=TRUE,trajectory_graph_segment_size =0.5,graph_label_size = 0.1,label_branch_points = FALSE)

plot_cells(mono_tumor,
           color_cells_by = "pseudotime",
           label_cell_groups=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE,alpha = 0.1,
           graph_label_size=0.1)






library(tidyverse)
#integrate TCGA data and perform lasso regression
matrix = read.csv('~/dk/thyroid/TCGA.THCA.sampleMap%2FHiSeqV2',header=TRUE,sep = '\t',row.names = 1)
head(colnames(matrix))
matrix = t(matrix)
rownames(matrix) = gsub('\\.','-',rownames(matrix))
gene = read.csv('~/dk/thyroid/network_matrx.txt',sep='\t')
head(gene)
matrix = matrix[,colnames(matrix) %in% gene$Row.names]
meta = read.csv('~/dk/thyroid/meta.txt',sep = '\t')
meta = meta %>% select('sample','OS','OS.time') %>% column_to_rownames(var = 'sample')
me = merge(meta,matrix,by = 'row.names')
me[1:4,1:4]
class(meta$OS)
me$OS.time = round(me$OS.time/12,digits = 0)
me = filter(me, !OS.time == 0)
rownames(me) = me$Row.names
library('survival')
library(glmnet)
x = as.matrix(me[,c(4:ncol(me))])
me$OS.time
y = data.matrix(Surv(me$OS.time,me$OS))
fit=glmnet(x,y,family = 'cox')
plot(fit)
cvfit = cv.glmnet(x,y,family = 'cox',type.measure = 'deviance',nfolds = 10)
plot(cvfit, xvar = 'lambda', label = TRUE)
cvfit$lambda.min
lambda.1se = cvfit$lambda.1se 
lambda.1se
library(survminer)
fit = survfit(Surv(OS.time,OS) ~ OS, data = me)
ggsurvplot(fit)
coefficient = coef(cvfit,s = cvfit$lambda.min)
coefficient
index = which(as.numeric(coefficient) != 0 )
index
indexcoff  = as.numeric(coefficient)[index]
indexcoff
rownames(coefficient)[index]